In [1]:
#This dictionary has each of the system numbers as the keys with the query numbers again as keys and the document number, rank and score as the value
#Example - {1:{1:[6567,1,5.0743]}}
dictir = {}

#This dictionary has each of the query numbers as keys with the document numbers and relevance as key value pairs
#Example - {1:{9090 :3}}
retr = {}


for i in range(1, 7):
    filename = "systems/S" + str(i) + ".results"
    with open(filename) as fp:
        for line in fp:
            tokens = line.split(" ")
            #If the system number is already not in
            if i not in dictir:
                dictir.update({i: {tokens[0] : [tokens[2], tokens[3], tokens[4]]}})
            else:
                dictir[i].update({tokens[0] : [tokens[2], tokens[3], tokens[4]]})

print(dictir)


{1: {'5': ['6660', '500', '1.3546'], '4': ['7309', '500', '0.7258'], '1': ['6259', '500', '1.2774'], '9': ['7120', '500', '2.1498'], '2': ['8124', '500', '1.5844'], '3': ['6936', '500', '0.8240'], '8': ['6259', '500', '1.2774'], '10': ['5492', '500', '1.0113'], '6': ['5017', '384', '1.1203'], '7': ['8732', '500', '1.5799']}, 2: {'5': ['6660', '500', '1.3546'], '4': ['7309', '500', '0.7258'], '1': ['6259', '500', '1.2774'], '9': ['7120', '500', '2.1498'], '2': ['8124', '500', '1.5844'], '3': ['6936', '500', '0.8240'], '8': ['6259', '500', '1.2774'], '10': ['5492', '500', '1.0113'], '6': ['5017', '384', '1.1203'], '7': ['8732', '500', '1.5799']}, 3: {'5': ['6660', '500', '1.3546'], '4': ['7309', '500', '0.7258'], '1': ['6259', '500', '1.2774'], '9': ['7120', '500', '2.1498'], '2': ['8124', '500', '1.5844'], '3': ['6936', '500', '0.8240'], '8': ['6259', '500', '1.2774'], '10': ['5492', '500', '1.0113'], '6': ['5017', '384', '1.1203'], '7': ['8732', '500', '1.5799']}, 4: {'5': ['6660', '50